<a href="https://colab.research.google.com/github/jaredmontierth/text-tuner/blob/main/fine_tuning_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

In [ ]:
torch.cuda.is_available()

True

In [ ]:
device = torch.device("cuda")
print(device)

cuda


In [ ]:
!pip install -U transformers
!pip install -U datasets


In [ ]:
from datasets import load_dataset
print("loading datasets")
dataset = load_dataset("cnn_dailymail", "3.0.0")
train_dataset = dataset["train"]
val_dataset = dataset["validation"]


In [ ]:
len(val_dataset)

In [ ]:
from transformers import BartTokenizer

model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)

def preprocess(example):
    
    input_text = example["article"]
    target_text = example["highlights"]

    input_encoding = tokenizer(input_text, max_length=1024, padding="max_length", truncation=True)
    target_encoding = tokenizer(target_text, max_length=256, padding="max_length", truncation=True)

    return {"input_ids": input_encoding["input_ids"],
            "attention_mask": input_encoding["attention_mask"],
            "labels": target_encoding["input_ids"]}

print("preprocessing")

train_data = train_dataset.map(preprocess, batched=True)
print("still preprocessing")
val_data = val_dataset.map(preprocess, batched=True)


In [ ]:
from transformers import BartForConditionalGeneration, TrainingArguments, Seq2SeqTrainer

print("setting model")
model = BartForConditionalGeneration.from_pretrained(model_name).to(device)

training_args = TrainingArguments(
    output_dir="./bart_summarization",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    evaluation_strategy="steps",
    eval_steps=5000,
    save_steps=5000,
    logging_steps=500,
    learning_rate=3e-5,
    warmup_steps=2000,
    weight_decay=0.01,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
)


In [ ]:
print("training model")
trainer.train()


Initialized empty Git repository in /content/.git/
